In [2]:


import pandas as pd # библиотека для работы с табличными данными
from rdkit import Chem # для работы с хим структурами
from rdkit.Chem import Descriptors # вычисляем мол вес


1. Загрузить очищенный датасет

Используйте результат мини‑таска 1 (датасет с валидными SMILES и активностями).

Будем работать с 2 датасетами, полученными из task1

In [3]:
df1 = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\no_duplicates_dataset.csv")
df2 = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\aggregated_dataset.csv")

2. Рассчитать дескрипторы

С помощью RDKit, Mordred, PaDEL или других библиотек извлеките молекулярные признаки для каждой молекулы.
Рекомендуется сохранить SMILES, standard_value и рассчитанные дескрипторы в одну таблицу.
В RDKit, например, можно получить 43 дескриптора: rdkit.Chem.Descriptors
В Mordred — более 1800, но часть из них может быть некорректной или пустой.

Поддерживаемые библиотеки
Вы можете использовать любую библиотеку для расчёта дескрипторов. Ниже приведены наиболее популярные инструменты:

Библиотека	Описание	Пример дескрипторов
RDKit	Стандартный инструментарий для химии	MW, LogP, TPSA, HBA/HBD
Mordred	Расширенный набор (1800+ дескрипторов)	Constitutional, Charge и др.
Padelpy	Python-обёртка над PaDEL-Descriptor	CDK, PubChem, MACCS, ECFP

Hint: попробуйте сформировать несколько разных наборов признаков — это поможет на следующем этапе сравнить производительность моделей.

Считаем с помощью RDKIT

In [ ]:
# функция для вычисления дескрипторов
def compute_rdkit_descriptors(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return [None] * len(rdkit_desc_list)
        return [desc_func(mol) for _, desc_func in Descriptors._descList]

# оставим в датасете колонки 'canonical_smiles', 'standard_value', и расчитаем для двух датасетов: с удаленными дубликатами, с агрегированными дубликатами
for i in [(df1, 'rdkit_descriptors_output_non_dublicates.csv'), (df2, 'rdkit_descriptors_output_agr.csv')]:
    df_input = i[0][['canonical_smiles', 'standard_value']].dropna().copy()

    # Список всех RDKit-дескрипторов
    rdkit_desc_list = [desc[0] for desc in Descriptors._descList]

    # Вычисляем дескрипторы
    df_input['rdkit_desc'] = df_input['canonical_smiles'].apply(compute_rdkit_descriptors)

    rdkit_desc_df = pd.DataFrame(df_input['rdkit_desc'].tolist(), columns=rdkit_desc_list)

    df_rdkit_final = pd.concat([df_input[['canonical_smiles', 'standard_value']], rdkit_desc_df], axis=1)
    df_rdkit_final.to_csv(i[1], index=False)


In [4]:
df1_desk_rdkit = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\rdkit_descriptors_output_non_dublicates.csv")
df2_desk_rdkit = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\rdkit_descriptors_output_agr.csv")
df1_desk_rdkit

,canonical_smiles,standard_value,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,11.568006,11.568006,0.338694,-3.165650,0.736620,11.545455,311.406,294.270,...,0,0,1,0,0,0,0,0,0,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,13.231678,13.231678,0.225436,-3.281084,0.667384,11.480000,357.406,341.278,...,0,0,1,0,0,0,0,0,0,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,13.531957,13.531957,0.252198,-3.293829,0.336163,11.500000,469.965,448.797,...,0,0,1,0,0,0,0,0,0,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,13.353649,13.353649,0.218100,-0.218100,0.558954,21.360000,335.397,316.245,...,0,0,0,0,0,0,0,0,0,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,13.224282,13.224282,0.274991,-3.216745,0.710290,11.625000,343.423,325.279,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5118,CC(=O)NCCc1cc(O)c(O)cc1C(=O)c1ccc(O)c(O)c1,768.00,12.618363,12.618363,0.107562,-0.501458,0.416407,10.375000,331.324,314.188,...,0,0,0,0,0,0,0,0,0,0
5119,CC(=O)NCCc1ccc(NC(=O)c2ccc(O)c(O)c2)c(O)c1,6860.00,12.099297,12.099297,0.104930,-0.535559,0.535039,10.208333,330.340,312.196,...,0,0,0,0,0,0,0,0,0,0
5120,CC(=O)N/C=C/c1ccc2c(c1)O[C@H](c1ccc(O)c(O)c1)[...,6430.00,11.569589,11.569589,0.181348,-0.824338,0.600501,17.928571,384.388,364.228,...,0,0,0,0,0,0,0,0,0,0
5121,COc1cc2c(cc1-c1c(C)noc1C)ncc1[nH]c(=O)n([C@H](...,100000.00,12.894516,12.894516,0.211833,-0.262700,0.473136,12.516129,415.453,394.285,...,0,0,0,0,0,0,0,0,0,0


Код для  padelpy долго работает, перезапущу, если будет недостаток в данных

In [ ]:


import pandas as pd
import os
import tempfile
from padelpy import padeldescriptor

# Параметры для padelpy
fingerprints = False          
d_2d = True                   
d_3d = False                  
removesalt = True             # Удаляем соли
log = True                   
threads = 4                   
maxruntime = 30000           
retainorder = True            # Сохраняем порядок
standardize_tautomers = True 

def compute_padel_for_df(df_with_ids, output_csv):

    # Создаём временный файл для SMILES
    with tempfile.NamedTemporaryFile(mode='w', suffix='.smi', delete=False) as tmp_file:
        tmp_path = tmp_file.name
        df_with_ids[['canonical_smiles', 'ID']].to_csv(tmp_path, sep=' ', index=False, header=False)

    try:
        # Запуск PaDEL
        padeldescriptor(
            mol_dir=tmp_path,
            d_file=output_csv,
            fingerprints=fingerprints,
            d_2d=d_2d,
            d_3d=d_3d,
            removesalt=removesalt,
            log=log,
            threads=threads,
            maxruntime=maxruntime,
            retainorder=retainorder,
            standardizetautomers=standardize_tautomers,
        )

        desc_df = pd.read_csv(output_csv)
        desc_df = desc_df.rename(columns={'Name': 'ID'})
        return desc_df

    finally:
        os.remove(tmp_path)

for df, out_csv in [
    (df1, 'padel_descriptors_non_duplicates.csv'),
    (df2, 'padel_descriptors_aggregated.csv'),
]:
    df_in = df[['canonical_smiles', 'standard_value']].dropna().reset_index(drop=True)
    df_in['ID'] = df_in.index.astype(str)
    df_desc = compute_padel_for_df(df_in, out_csv)
    df_final = pd.merge(df_in, df_desc, on='ID')
    df_final.drop(columns=['ID'], inplace=True)
    df_final.to_csv(out_csv, index=False)


С mordred были проблемы, не работало с текущей версией numpy, созданная среда с нужной версией не помогла, возникали ошибки типа DLL. Было принято решение пофиксить ошибки в самой библиотеке, заменив функици из старой версии numpy на новые.

In [5]:
from rdkit import Chem
from mordred import Calculator, descriptors
import pandas as pd

# игнорируем 3d дискрипторы
calc = Calculator(descriptors, ignore_3D=True)

dataframes = [
    (df1, 'mordred_descriptors_output_non_duplicates.csv'),
    (df2, 'mordred_descriptors_output_aggregated.csv')
]

def compute_mordred(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol)
    except Exception:
        return None

for df_source, output_file in dataframes:
    df_input = df_source[['canonical_smiles', 'standard_value']].dropna().copy()
    
    # Вычисляем дескрипторы 
    mordred_results = df_input['canonical_smiles'].apply(compute_mordred)

    mordred_df = pd.DataFrame(
        [res if res is not None else [None]*len(calc.descriptors) for res in mordred_results],
        columns=[str(d) for d in calc.descriptors]
    )

    df_mordred_final = pd.concat([df_input[['canonical_smiles', 'standard_value']], mordred_df], axis=1)
    
    df_mordred_final.to_csv(output_file, index=False)


c:\Users\Redmi\miniconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\Redmi\miniconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


3. (Опционально) Рассчитать фингерпринты


Сформируйте бинарные векторы (например, Morgan/ECFP, MACCS), которые можно использовать как альтернативу дескрипторам.
Фингерпринты могут пригодиться для сравнения моделей и оценки важности признаков.

In [ ]:
from rdkit import Chem, DataStructs                    # работа с молекулами и преобразование фингерпринтов
from rdkit.Chem import Descriptors, AllChem, MACCSkeys # расчет дескрипторов, Morgan-фингерпринтов и MACCS-ключей
import numpy as np                                     # массивы


# Функция генерации Morgan-фингерпринта (ECFP) из SMILES
def get_morgan_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles) 
    if mol is None:                   
        return [None] * nBits
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)  # Вычисляем фингерпринт
    arr = np.zeros((nBits,), dtype=int)  
    DataStructs.ConvertToNumpyArray(fp, arr)  
    return arr.tolist() 


# Функция генерации MACCS-фингерпринта
def get_maccs_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * 167  # MACCS содержит 167 бит, включая неиспользуемый 0-й
    fp = MACCSkeys.GenMACCSKeys(mol)
    arr = np.zeros((167,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr.tolist()

for df, output_file in [(df1, 'output_non_duplicates_fing.csv'),
                        (df2, 'output_aggregated_fing.csv')]:
    
    df_input = df[['canonical_smiles', 'standard_value']].dropna().copy()
        # Вычисляем Morgan фингерпринты
    df_input['morgan_fp'] = df_input['canonical_smiles'].apply(get_morgan_fp)
    morgan_df = pd.DataFrame(df_input['morgan_fp'].tolist(), columns=[f'Morgan_{i}' for i in range(2048)])

    # Вычисляем MACCS фингерпринты
    df_input['maccs_fp'] = df_input['canonical_smiles'].apply(get_maccs_fp)
    maccs_df = pd.DataFrame(df_input['maccs_fp'].tolist(), columns=[f'MACCS_{i}' for i in range(167)])
    maccs_df = maccs_df.drop(columns=['MACCS_0'])  # Удаляем колонку MACCS_0

    df_final = pd.concat(
        [df_input[['canonical_smiles', 'standard_value']],
         rdkit_desc_df, morgan_df, maccs_df],
        axis=1.)

    df_final.to_csv(output_file, index=False)


[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerator
[20:08:08] DEPRECATION WARNING: please use MorganGenerat

4. Отфильтровать и отобрать признаки

Удалите дескрипторы с:
некорректными или отсутствующими значениями (NaN);
нулевой дисперсией (одинаковое значение для всех молекул);
высокой корреляцией (например, r > 0.7 между признаками — можно оставить один из пары);
Также рекомендуется исключить категориальные признаки, если они неинформативны или дублируют числовые.
Внимание: нормализация и стандартизация признаков не требуется на этом этапе — она будет выполнена в следующем мини‑таске

In [3]:
import pandas as pd
import numpy as np

def remove_ambiguous_relations(df, ref):

    # Оставляем только строки с relation == '<' или '>'
    ambiguous = ref[
        ref['standard_relation'].isin(['<', '>'])
    ][['canonical_smiles', 'standard_value']]

    # Удалим строки из текущего df, если совпадает smiles и standard_value
    cleaned_df = df.merge(
        ambiguous,
        on=['canonical_smiles', 'standard_value'],
        how='left',
        indicator=True
    ).query('_merge == "left_only"').drop(columns=['_merge'])

    return cleaned_df

no_duplicates_dataset = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\no_duplicates_dataset.csv")
aggregated_dataset = pd.read_csv(r"C:\Users\Redmi\Desktop\Биология\хакатон\aggregated_dataset.csv")

dataframes = [
    ('clean_mordred_descriptors_output_non_duplicates.csv', 'mordred_descriptors_output_non_duplicates.csv'),
    ('clean_mordred_descriptors_output_aggregated.csv', 'mordred_descriptors_output_aggregated.csv'),
    ('clean_rdkit_descriptors_output_non_dublicates.csv', 'rdkit_descriptors_output_non_dublicates.csv'), 
    ('clean_rdkit_descriptors_output_agr.csv', 'rdkit_descriptors_output_agr.csv')]
for output_file, df_name in dataframes:
    df = pd.read_csv(df_name) 
    
    
    # Удаляем признаки с NaN или бесконечностями
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    print(df_clean.shape)
    if 'duplicates' in df_name:

        df_clean = remove_ambiguous_relations(df_clean, no_duplicates_dataset)
    else:
        df_clean = remove_ambiguous_relations(df_clean, aggregated_dataset)
    print(df_clean.shape)
    # удляем столбцы с 0 дисперсией
    df_clean = df_clean.loc[:, df_clean.nunique() > 1]

    # Удаляем признаки с высокой корреляцией |r| > 0.7
    def remove_highly_correlated(df, threshold=0.7):
        corr_matrix = df.corr().abs()
        # элементы над диагональю
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
        return df.drop(columns=to_drop)
    

    # Оставим только числовые столбцы
    df_clean = df_clean.select_dtypes(include=[np.number])
    df_clean = remove_highly_correlated(df_clean, threshold=0.7)
    print(f"Осталось признаков в «{output_file}»: {df_clean.shape[1]}")
    df_clean.to_csv(output_file, index=False)


C:\Users\Redmi\AppData\Local\Temp\ipykernel_30952\2235297854.py:30: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,55,56,57,58,59,60,61,62,63,143,144,152,153,154,155,156,157,158,159,160,161,162,170,171,179,180,188,189,197,198,206,207,215,216,224,225,233,234,235,236,237,238,239,240,241,242,243,262,263,264,265,266,267,268,269,270,343,344,345,346,347,348,349,350,351,359,360,368,369,370,371,372,373,374,375,376,377,378,386,387,395,396,404,405,413,414,422,423,431,432,440,441,449,450,451,452,453,454,455,456,457,458,465,466,473,474,475,476,477,478,479,480,481,482,489,490,497,498,505,506,513,514,521,522,529,530,537,538,545,546,547,548,549,550,551,552,553,554,561,562,569,570,571,572,573,574,575,576,577,578,585,586,593,594,601,602,609,610,617,618,625,626,633,634,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,69

(5123, 1615)
(4204, 1615)
Осталось признаков в «clean_mordred_descriptors_output_non_duplicates.csv»: 173


C:\Users\Redmi\AppData\Local\Temp\ipykernel_30952\2235297854.py:30: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,55,56,57,58,59,60,61,62,63,143,144,152,153,154,155,156,157,158,159,160,161,162,170,171,179,180,188,189,197,198,206,207,215,216,224,225,233,234,235,236,237,238,239,240,241,242,243,262,263,264,265,266,267,268,269,270,343,344,345,346,347,348,349,350,351,359,360,368,369,370,371,372,373,374,375,376,377,378,386,387,395,396,404,405,413,414,422,423,431,432,440,441,449,450,451,452,453,454,455,456,457,458,465,466,473,474,475,476,477,478,479,480,481,482,489,490,497,498,505,506,513,514,521,522,529,530,537,538,545,546,547,548,549,550,551,552,553,554,561,562,569,570,571,572,573,574,575,576,577,578,585,586,593,594,601,602,609,610,617,618,625,626,633,634,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,69

(5123, 1615)
(4205, 1615)
Осталось признаков в «clean_mordred_descriptors_output_aggregated.csv»: 173
(5123, 207)
(4235, 207)
Осталось признаков в «clean_rdkit_descriptors_output_non_dublicates.csv»: 112
(5123, 207)
(4205, 207)
Осталось признаков в «clean_rdkit_descriptors_output_agr.csv»: 112
